## scripts to evaluate matching results (antibody drop version)

calculation of matching accuracy, foscknn, foscttm and retrieve slt ari f1 scores.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
sys.path.append("./")
import metrics
import utils

In [4]:
out_dir = "/bench_test3/output/drop/"
ann_list1 = []
ann_list2 = []
foscttm_list = []
tmp = []
knn_tmp = []
slt_f1 = []
ari_f1 = []

for i in range(5):
    batch = 'b'+str(i+1)+'/'
    for j in range(4):
        
        drop_name = "dropLV"+str(j)+"/"
        root_dir = out_dir+batch+drop_name
        # produce liger matching
        lgx = pd.read_csv(root_dir+"lgunimf/full_embed_x0.csv")
        lgy = pd.read_csv(root_dir+"lgunimf/full_embed_y0.csv")
        lg_dist = utils.cdist_correlation(lgy.to_numpy(), lgx.to_numpy())
        lg_full_match, lg_scores = metrics.get_knn_matching(lg_dist)
        lgmatch = [lg_full_match,np.arange(lgy.shape[0]),lg_scores]
        # produce bsc matching
        bscx = pd.read_csv(root_dir+"bsc/full_embed_x0.csv")
        bscy = pd.read_csv(root_dir+"bsc/full_embed_y0.csv")
        bsc_dist = utils.cdist_correlation(bscy.to_numpy(), bscx.to_numpy())
        bsc_full_match, bsc_scores = metrics.get_knn_matching(bsc_dist)
        bscmatch = [bsc_full_match,np.arange(bscx.shape[0]),bsc_scores]
        # harmony matching
        hmx = pd.read_csv(root_dir+"hm/full_embed_x0.csv")
        hmy = pd.read_csv(root_dir+"hm/full_embed_y0.csv")
        hm_dist = utils.cdist_correlation(hmy.to_numpy(), hmx.to_numpy())
        hm_full_match, hm_scores = metrics.get_knn_matching(hm_dist)
        hmmatch = [hm_full_match,np.arange(hmx.shape[0]),hm_scores]
        # load mf matching
        mf = pd.read_csv(root_dir+"mf/full_idx.csv")
        mfmatch = [mf['idx1'].tolist(),mf['idx2'].tolist(),mf['score'].tolist()]
        # load sr matching
        #sr = pd.read_csv(root_dir+"sr/full_idx.csv")
        #srmatch = [sr['idx1'].tolist(),sr['idx2'].tolist(),sr['score'].tolist()]
        srx = pd.read_csv(root_dir+"sr/full_embed_x0.csv")
        sry = pd.read_csv(root_dir+"sr/full_embed_y0.csv")
        sr_dist = utils.cdist_correlation(sry.to_numpy(), srx.to_numpy())
        sr_full_match, sr_scores = metrics.get_knn_matching(sr_dist)
        srmatch = [sr_full_match,np.arange(sry.shape[0]),sr_scores]
        # load embedding too
        # mf embed
        mfx = pd.read_csv(root_dir+"mf/full_embed_x0.csv")
        mfy = pd.read_csv(root_dir+"mf/full_embed_y0.csv")
        # sr embed
        #srx = pd.read_csv(root_dir+"sr/full_embed_x0.csv")
        #sry = pd.read_csv(root_dir+"sr/full_embed_y0.csv")
        # read meta info
        temp_dir = '/home/bkzhu/super_mario/bench_test3/input/'
        meta = pd.read_csv(temp_dir + batch+ 'meta.csv')
        annotationlv1 = meta['celltype.l1'].to_numpy()
        order = (2, 1)
        acc_ann_hm = metrics.get_matching_acc(
                matching=hmmatch, 
                labels1=annotationlv1, 
                labels2=annotationlv1,
                order = order
            )

        acc_ann_lg = metrics.get_matching_acc(
                matching=lgmatch, 
                labels1=annotationlv1, 
                labels2=annotationlv1,
                order = order
            )

        acc_ann_sr = metrics.get_matching_acc(
                matching=srmatch, 
                labels1=annotationlv1, 
                labels2=annotationlv1,
                order = order
            )

        acc_ann_mf = metrics.get_matching_acc(
                matching=mfmatch, 
                labels1=annotationlv1, 
                labels2=annotationlv1,
                order = order
            )
        
        acc_ann_bsc = metrics.get_matching_acc(
            matching=bscmatch, 
            labels1=annotationlv1, 
            labels2=annotationlv1,
            order = order
        )
        ann_list1.extend([acc_ann_mf, acc_ann_sr, acc_ann_lg, acc_ann_hm, acc_ann_bsc])
        #
        annotationlv2 = meta['celltype.l2'].to_numpy()
        order = (2, 1)
        acc_ann_hm = metrics.get_matching_acc(
                matching=hmmatch, 
                labels1=annotationlv2, 
                labels2=annotationlv2,
                order = order
            )

        acc_ann_lg = metrics.get_matching_acc(
                matching=lgmatch, 
                labels1=annotationlv2, 
                labels2=annotationlv2,
                order = order
            )

        acc_ann_sr = metrics.get_matching_acc(
                matching=srmatch, 
                labels1=annotationlv2, 
                labels2=annotationlv2,
                order = order
            )
        acc_ann_mf = metrics.get_matching_acc(
                matching=mfmatch, 
                labels1=annotationlv2, 
                labels2=annotationlv2,
                order = order
            )
        acc_ann_bsc = metrics.get_matching_acc(
            matching=bscmatch, 
            labels1=annotationlv2, 
            labels2=annotationlv2,
            order = order
        )
        ann_list2.extend([acc_ann_mf, acc_ann_sr, acc_ann_lg, acc_ann_hm, acc_ann_bsc])
        # foscttm
        # mf
        mfdist = utils.cdist_correlation(mfx.to_numpy(), mfy.to_numpy())
        mf_fos = metrics.get_foscttm(mfdist)
        # sr
        srdist = utils.cdist_correlation(srx.to_numpy(), sry.to_numpy())
        sr_fos = metrics.get_foscttm(srdist)
        # lg
        lg_fos = metrics.get_foscttm(lg_dist)
        # hm
        hm_fos = metrics.get_foscttm(hm_dist)
        # bsc
        bsc_fos = metrics.get_foscttm(bsc_dist)
        foscttm_list.extend([mf_fos,sr_fos,lg_fos,hm_fos,bsc_fos])
        # single-cell level accuracy
        mfsc = metrics.get_matching_alignment_score(mfmatch,n_samples=lgy.shape[0])
        srsc = metrics.get_matching_alignment_score(srmatch,n_samples=lgy.shape[0])
        lgsc = metrics.get_matching_alignment_score(lgmatch,n_samples=lgy.shape[0])
        hmsc = metrics.get_matching_alignment_score(hmmatch,n_samples=lgy.shape[0])
        bscsc = metrics.get_matching_alignment_score(bscmatch,n_samples=lgy.shape[0])
        tmp.extend([mfsc,srsc,lgsc,hmsc,bscsc])
        # knn search tmp
        k = 100
        knn_match_mf = metrics.get_knn_alignment_score(
            dist=mfdist,
            k_max=k
        )
        knn_match_sr = metrics.get_knn_alignment_score(
            dist=srdist,
            k_max=k
        )
        knn_match_lg = metrics.get_knn_alignment_score(
            dist=lg_dist,
            k_max=k
        )
        knn_match_hm = metrics.get_knn_alignment_score(
            dist=hm_dist,
            k_max=k
        )
        knn_match_bsc = metrics.get_knn_alignment_score(
            dist=bsc_dist,
            k_max=k
        )
        # store the knn tmp
        knn_tmp.extend([knn_match_mf.tolist(), knn_match_sr.tolist(),
                        knn_match_lg.tolist(), knn_match_hm.tolist(), knn_match_bsc.tolist()])
        # also need to read the integration metrics
        mf_met = pd.read_csv(root_dir+"mf/metrics.csv")
        sr_met = pd.read_csv(root_dir+"sr/metrics.csv")
        lg_met = pd.read_csv(root_dir+"lg/metrics.csv")
        hm_met = pd.read_csv(root_dir+"hm/metrics.csv")

        slt_f1.extend([mf_met.loc[0,'slt_f1'],sr_met.loc[0,'slt_f1'],lg_met.loc[0,'slt_f1'],hm_met.loc[0,'slt_f1']])
        ari_f1.extend([mf_met.loc[0,'ari_f1'],sr_met.loc[0,'ari_f1'],lg_met.loc[0,'ari_f1'],hm_met.loc[0,'ari_f1']])

/home/bkzhu/super_mario/bench_test3/scripts_benchmark/metrics.py:98: RuntimeWarning: invalid value encountered in true_divide
  indices2_and_scores[1] = list(np.array(indices2_and_scores[1]) / np.sum(indices2_and_scores[1]))


In [5]:
# construct batch
b = np.array(["b1","b2","b3","b4","b5"])
binfo = np.repeat(b, [20,20,20,20,20], axis=0)
m = ["mf","sr","lg","hm","bsc"]
minfo = m*5*4

drop = ["dropLv0","dropLv1","dropLv2","dropLv3"]
dropinfo = np.repeat(drop, [5,5,5,5], axis=0).tolist()
dropinfo = dropinfo*5

data = {'batch':binfo, 'method':minfo,'drop':dropinfo,
       'ann1':ann_list1, 'ann2':ann_list2, 'foscttm':foscttm_list, 'tmp':tmp }

matching_result = pd.DataFrame(data)

In [7]:
matching_result.to_csv("/bench_test3/output/drop4_batch5_resultsV2.csv")

In [8]:
# construct batch
b = np.array(["b1","b2","b3","b4","b5"])
binfo = np.repeat(b,[100*4*5,100*4*5,100*4*5,100*4*5,100*4*5], axis=0).tolist()

m = np.array(["mf", "sr","lg","hm","bsc"])
minfo = np.repeat(m, [100,100,100,100,100], axis=0).tolist()
minfo2 = minfo * 4 * 5

knn = [item for sublist in knn_tmp for item in sublist]
step = [i for i in range(100)]*4*5*5

drop = ["dropLv0","dropLv1","dropLv2","dropLv3"]
dropinfo = np.repeat(drop, [100*5,100*5,100*5,100*5], axis=0).tolist()
dropinfo = dropinfo*5

data = {'batch':binfo, 'method':minfo2,'knn_tmp': knn, 'step':step, 'drop': dropinfo}

knn_result = pd.DataFrame(data)

In [10]:
# save the results as csvs
knn_result.to_csv("/bench_test3/output/drop4_batch5_knntmpV2.csv")